In [ ]:
#imports 
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry, SamPredictor
import os
from ultralytics import YOLO
import pandas as pd

In [ ]:
# Set the path to the folder containing your dataset
# Adjust this path to where you saved the images
dataset_folder = 'DS203-2025-S1-E7-project-images/images/'

# We will store the loaded images in a list
loaded_images = []
# It's also useful to store their filenames
image_filenames = []

print(f"Starting to load images from: {dataset_folder}")

# Loop through every file in the folder
for filename in os.listdir(dataset_folder):
    
    # Check if the file is an image (e.g., .jpg, .png)
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        
        # Create the full path to the image
        image_path = os.path.join(dataset_folder, filename)
        
        # Load the image using OpenCV
        image = cv2.imread(image_path)
        
        if image is not None:
            # If loading was successful, add it to our list
            loaded_images.append(image)
            image_filenames.append(filename)
        else:
            print(f"Warning: Could not load {filename}")

print(f"---")
print(f"Success! Loaded {len(loaded_images)} images.")
print(f"Example: The first image '{image_filenames[0]}' has dimensions: {loaded_images[0].shape}")

In [ ]:
def process_image(image):
    """
    Combines all preprocessing and splitting into one function.
    
    1. Applies all project rules (crop, no-upscale, downscale).
    2. If valid, splits the 800x600 image into an 8x8 grid.
    
    Returns: 
        - A list of 64 (100x75) image cells if successful.
        - None if the image is discarded.
    """
    
    # --- Part 1: Processing (Same as before) ---
    TARGET_WIDTH = 800
    TARGET_HEIGHT = 600
    TARGET_ASPECT = TARGET_WIDTH / TARGET_HEIGHT
    
    h, w = image.shape[:2]
    original_aspect = w / h
    
    processed_image = image
    
    if abs(original_aspect - TARGET_ASPECT) > 0.01:
        if original_aspect > TARGET_ASPECT:
            new_width = int(h * TARGET_ASPECT)
            left = (w - new_width) // 2
            right = left + new_width
            processed_image = image[:, left:right]
        else:
            new_height = int(w / TARGET_ASPECT)
            top = (h - new_height) // 2
            bottom = top + new_height
            processed_image = image[top:bottom, :]
            
    h, w = processed_image.shape[:2]
    
    if w < TARGET_WIDTH or h < TARGET_HEIGHT:
        return None  # Discard
        
    if w > TARGET_WIDTH or h > TARGET_HEIGHT:
        processed_image = cv2.resize(processed_image, 
                                     (TARGET_WIDTH, TARGET_HEIGHT), 
                                     interpolation=cv2.INTER_AREA)

    # At this point, 'processed_image' is a valid 800x600 image.

    return processed_image

In [ ]:
print(f"Loaded {len(loaded_images)} images. Now processing and filtering...")
final_images = []
final_filenames = []

# Use zip() to loop over both lists at the same time
for (image, filename) in zip(loaded_images, image_filenames):
    
    # Process the imag
# Now, 'final_grids[i]' is the list of 64 cells
# that belongs to 'final_filenames[i]'e using your combined function
    final_image = process_image(image)
    
    # --- This is the critical check ---
    # Only keep the grid AND its filename if processing was successful
    if final_image is not None:
        final_images.append(final_image)
        final_filenames.append(filename)

print("---")
print("Processing complete.")
print(f"Kept {len(final_filenames)} valid images for training.")

In [ ]:
print(final_filenames)

In [ ]:
detector = YOLO('yolov8n.pt') 
MODEL_TYPE = "vit_b"  # or "vit_b"
CHECKPOINT_PATH = "sam_vit_b_01ec64.pth" # Your downloaded model path
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH)
sam.to(device=DEVICE)
predictor = SamPredictor(sam)
OUTPUT_FOLDER = 'Yolo_SAM'
out_dir = os.path.join(OUTPUT_FOLDER)
os.makedirs(out_dir, exist_ok=True)

In [ ]:
def detect_box(image_bgr, conf_thresh=0.3):
    results = detector(image_bgr, verbose=False)
    input_boxes = []
    if results and results[0].boxes:  # Check if any boxes were detected
        boxes = results[0].boxes
        coords = boxes.xyxy.cpu().numpy()    # shape (N,4)
        confs = boxes.conf.cpu().numpy()     # shape (N,)
        cls_ids = boxes.cls.cpu().numpy().astype(int)  # shape (N,)

        for i in range(len(confs)):
            input_box = coords[i]
            class_id = int(cls_ids[i])
            confidence = float(confs[i])
            class_name = detector.names.get(class_id, str(class_id))
            if confidence >= conf_thresh:
                input_boxes.append(input_box)
        if input_boxes :
            return input_boxes
        else:
            print(len(results[0].boxes))
            print("No Boxes found with thres > 0.35")
            return None
    else:
        print("❌ No objects detected by YOLOv8.")
        return None

In [ ]:
def mask_image(image_bgr, image_path):
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    # Set the image for the predictor
    predictor.set_image(image_rgb)
    input_boxes = detect_box(image_bgr)
    if not input_boxes:
        print("No boxes to mask.")
        return None

    combined_mask = None
    for box in input_boxes:
        masks, scores, logits = predictor.predict(
            box=box,                # provide a single box (shape (4,))
            multimask_output=False
        )
        mask = masks[0]  # boolean mask for this box
        if combined_mask is None:
            combined_mask = mask.copy()
        else:
            combined_mask |= mask     # union of masks

    mask_uint8 = combined_mask.astype(np.uint8) * 255
    mask_color = cv2.cvtColor(mask_uint8, cv2.COLOR_GRAY2BGR)
    mask_bool = combined_mask.astype(bool)

    # Option A: Color tint overlay (BGR color, here red)
    color = np.array([0, 0, 255], dtype=np.uint8)  # BGR
    alpha = 0.6  # overlay strength on animal region (0..1)

    colored = np.zeros_like(image_bgr, dtype=np.uint8)
    colored[:] = color

    # Blend only where mask is True
    blended = image_bgr.copy()
    blended[mask_bool] = (image_bgr[mask_bool].astype(np.float32) * (1 - alpha) +
                          color.astype(np.float32) * alpha).astype(np.uint8)

    # Save visuals for inspection
    out_name= f"{image_path}_mask"
    out_path = os.path.join(out_dir, out_name)
    cv2.imwrite(out_path, blended)
    return combined_mask

In [ ]:
masks = {}
count = 0
for image, filepath in zip(final_images, final_filenames):
    count += 1
    print(f"processing image {count}: {filepath}")
    masks[filepath] = mask_image(image, filepath)


In [ ]:
def detect_label_in_cell(mask):
    """
    Returns 1 if >=80% of pixels in `mask` are True/foreground, else 0.
    Accepts boolean mask or 0/1 uint8 array of any shape.
    """
    mask_bool = np.asarray(mask).astype(bool)
    total_pixels = mask_bool.size
    total_wildlife_pixels = int(mask_bool.sum())
    return 1 if (total_wildlife_pixels / total_pixels) >= 0.2 else 0

In [ ]:
import pandas as pd 
final_df = pd.DataFrame(columns=["cell_Image_path", "Image_path","label"])
final_grid = []
final_label = []
CELL_OUTPUT_FOLDER = 'Cell_Images'
cells_dir = os.path.join(CELL_OUTPUT_FOLDER)
os.makedirs(cells_dir, exist_ok=True)

In [ ]:

TARGET_WIDTH = 800
TARGET_HEIGHT = 600
def make_grid(image,image_mask,path):
    GRID_ROWS = 8
    GRID_COLS = 8
    # Calculate the size of each cell
    # 800 / 8 = 100 pixels wide
    # 600 / 8 = 75 pixels high
    cell_height = TARGET_HEIGHT // GRID_ROWS  # 75
    cell_width = TARGET_WIDTH // GRID_COLS   # 100
    n=1
    # Loop through the rows and columns
    for r in range(GRID_ROWS):
        for c in range(GRID_COLS):
            
            # Calculate the pixel coordinates for the slice
            y1 = r * cell_height
            y2 = y1 + cell_height
            x1 = c * cell_width
            x2 = x1 + cell_width
            
            # Use NumPy slicing to extract the cell
            # [y1:y2, x1:x2]
            cell = image[y1:y2, x1:x2].copy()
            if image_mask is None:
                mask = np.zeros((y2 - y1, x2 - x1), dtype=bool)
            else:
                m = np.asarray(image_mask)
                if m.ndim == 3:
                    m = m[..., 0]
                # ensure boolean and correct shape
                mask = m[y1:y2, x1:x2].astype(bool)
            label = detect_label_in_cell(mask)
            final_grid.append(cell)
            final_label.append(label)
            base = os.path.basename(path)
            name_no_ext = os.path.splitext(base)[0]
            cell_id = f"{name_no_ext}_{n}"
            cell_image_path = os.path.join(CELL_OUTPUT_FOLDER, cell_id + ".jpg")
            final_df.loc[len(final_df)] = [cell_id + ".jpg", name_no_ext,int(label)]
            cv2.imwrite(cell_image_path, cell)
            n=n+1
    return None

In [ ]:
for image,filename in zip(final_images,final_filenames):
    mask = masks[filename]
    make_grid(image,mask,filename)

In [ ]:
final_df.to_csv("final_labels.csv", index=False)